# Load Required Library




In [ ]:
import pandas as pd
import os
import json
import re
import numpy as np
import random
import string
import pickle
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import gensim
from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

!pip install underthesea
from underthesea import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 7.5MB 8.7MB/s 
     |████████████████████████████████| 747kB 36.4MB/s 
     |████████████████████████████████| 753.2MB 10kB/s 
     |████████████████████████████████| 245kB 7.6MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 1.3MB 34.2MB/s 
     |████████████████████████████████| 1.1MB 36.0MB/s 
     |████████████████████████████████| 901kB 27.6MB/s 
     |████████████████████████████████| 2.9MB 35.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=3fb6fcddc74fbcefe9b37684d3dbcc326c1fb7f3983f602a20afddc8844746b7
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqe

# Data Preparation




## 1.Read Raw Data




In [ ]:
import zipfile
with zipfile.ZipFile("DataChatbot.zip","r") as zf:
    zf.extractall('DataChatbot')
print(zf)

<zipfile.ZipFile [closed]>


In [ ]:
data=[]
label=[]
error=0
import glob
import os
path = 'DataChatbot'
for filename in glob.glob(os.path.join(path, '*.txt')):
   with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
      for line in f:
        part = line.strip().strip('__eou_').split('__eou__')
        data.append(", ".join(part[:-1]))
        label.append(part[-1])
print('question',data[:5])
print('Aws',label[:5])

question ['Nghe nói thông mình lắm  phải không? ', 'Robot ơi tui thất tình rồi, bạn có thể làm bạn gái tui không ? ', 'Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu vậy ? ', 'Robot có thích chơi búp bê không ? ', 'Theo robot thì 1 cộng 1 bằng mấy ? ']
Aws [' Hơn m được rồi phải không ', ' Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đi nhé, bạn rất tốt nhưng tui rất tiếc ', ' Mình không ghi được vì vô cực tràn bộ nhớ ', ' Mình chẳng những chơi búp bê mà còn quất khủng long siêu nhân gao ', ' Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón ']


## 2.Data Preprocessing

In [ ]:
df = pd.DataFrame(data)
df.columns = ["Question"]
df['Aws']=df['Question'].apply(len)
df['Aws'] = pd.DataFrame(label)
print(df)

                                               Question                                                Aws
0                 Nghe nói thông mình lắm  phải không?                          Hơn m được rồi phải không 
1     Robot ơi tui thất tình rồi, bạn có thể làm bạn...   Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...
2     Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...         Mình không ghi được vì vô cực tràn bộ nhớ 
3                   Robot có thích chơi búp bê không ?    Mình chẳng những chơi búp bê mà còn quất khủn...
4                   Theo robot thì 1 cộng 1 bằng mấy ?     Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón 
...                                                 ...                                                ...
5850          Con gà có trước hay quả trứng có trước ?                                   Kệ nó hỏi làm gì 
5851          Con chó có trước hay quả trứng có trước?                                 Quả trứng có trước 
5852   Nhiệt độ nóng chảy của thủy ng

In [ ]:
def pre_process(questions):
    stop_words = stopwords.words("english")
    
    # Remove non english words
    questions = [re.sub('[^a-z(c++)(c#)]', ' ', x.lower()) for x in questions]
    # Tokenlization
    questions_tokens = [nltk.word_tokenize(t) for t in questions]
    # Removing Stop Words
    questions_stop = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)]
                      for tokens in questions_tokens]
    
    questions_stop = pd.Series(questions_stop)
    return questions_stop

In [ ]:
# Initial preprocessing training data
# Initial preprocessing training data
questions = df['Question']
questions_pp = pre_process(questions)
df

,Question,Aws
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón"
...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?


In [ ]:
df['Question_Tokens']=df['Question'].apply(len)

## 3.Text Classification using underthesea

In [ ]:
from underthesea import classify
questions = df['Question']
for i in range(len(df['Question'])):
  df['Question_Tokens'][i]=classify(df['Question'][i])
df

2021-04-25 02:10:21,559 https://github.com/undertheseanlp/playground/releases/download/1.3.x/tc_svm_vntc_20201228.zip not found in cache, downloading to /tmp/tmp7o6j5pu0


100%|██████████| 75578053/75578053 [00:01<00:00, 49820813.30B/s]

2021-04-25 02:10:23,353 copying /tmp/tmp7o6j5pu0 to cache at /root/.underthesea/models/tc_svm_vntc_20201228.zip


2021-04-25 02:10:23,452 removing temp file /tmp/tmp7o6j5pu0


,Question,Aws,Question_Tokens
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi]
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song]
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc]
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc]
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc]
...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi]
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song]
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song]
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song]


In [ ]:
data_tokens = pd.DataFrame({'Question': list(df['Question']),
                            'Question_Tokens': list(df['Question_Tokens']),
                            'Answer': list(df['Aws'])
                           })
data_tokens.head()

,Question,Question_Tokens,Answer
0,Nghe nói thông mình lắm phải không?,[chinh_tri_xa_hoi],Hơn m được rồi phải không
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...",[doi_song],"Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,[khoa_hoc],Mình không ghi được vì vô cực tràn bộ nhớ
3,Robot có thích chơi búp bê không ?,[khoa_hoc],Mình chẳng những chơi búp bê mà còn quất khủn...
4,Theo robot thì 1 cộng 1 bằng mấy ?,[khoa_hoc],"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón"


In [ ]:
data_example = pd.DataFrame(data_tokens['Question'])
length = data_example['Question'].apply(len)
data_example = data_example.assign(Question_Length=length)
data_example.head()

,Question,Question_Length
0,Nghe nói thông mình lắm phải không?,37
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...",63
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,53
3,Robot có thích chơi búp bê không ?,35
4,Theo robot thì 1 cộng 1 bằng mấy ?,35


In [ ]:
df.columns=['Question','Answer','Question_Tokens']
df

,Question,Answer,Question_Tokens
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi]
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song]
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc]
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc]
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc]
...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi]
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song]
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song]
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song]


In [ ]:
df['rs_result']=df['Question'].apply(len)
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],37
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],63
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],53
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],35
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],35
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],41
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],41
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],48
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],85


In [ ]:
# Raw data
for i in range(len(df['Question'])):
  example = data_example['Question'][i]
  raw_title = 'Raw Data'
  raw_result = example
  df['rs_result'][i]=raw_result
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],Nghe nói thông mình lắm phải không?
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],"Robot ơi tui thất tình rồi, bạn có thể làm bạn..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],Robot có thích chơi búp bê không ?
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],Theo robot thì 1 cộng 1 bằng mấy ?
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],Con gà có trước hay quả trứng có trước ?
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],Con chó có trước hay quả trứng có trước?
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch..."


In [ ]:
for i in range(len(df['Question'])):
  re_result=[]
  re_result.append(df['rs_result'][i])
  df['rs_result'][i]=re_result
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],[Nghe nói thông mình lắm phải không? ]
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],"[Robot ơi tui thất tình rồi, bạn có thể làm bạ..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],[Robot cho mình hỏi IQ của bạn chỉ số bao nhiê...
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],[Robot có thích chơi búp bê không ? ]
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],[Theo robot thì 1 cộng 1 bằng mấy ? ]
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],[Con gà có trước hay quả trứng có trước ? ]
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],[Con chó có trước hay quả trứng có trước? ]
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],[Nhiệt độ nóng chảy của thủy ngân là bao nhiêu...
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],"[Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 c..."


## 4.Word Segmentation using underthesea 

In [ ]:
for i in range(len(df['Question'])):
  tk_result = [word_tokenize(t) for t in df['rs_result'][i]]
  df['rs_result'][i]=tk_result
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],"[[Nghe, nói, thông, mình, lắm, phải, không, ?]]"
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],"[[Robot, ơi, tui, thất tình, rồi, ,, bạn, có t..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],"[[Robot, cho, mình, hỏi, IQ, của, bạn, chỉ số,..."
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],"[[Robot, có, thích, chơi, búp bê, không, ?]]"
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],"[[Theo, robot, thì, 1, cộng, 1, bằng, mấy, ?]]"
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],"[[Con, gà, có, trước, hay, quả, trứng, có, trư..."
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],"[[Con, chó, có, trước, hay, quả, trứng, có, tr..."
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],"[[Nhiệt độ, nóng chảy, của, thủy ngân, là, bao..."
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],"[[Cái, gì, chỉ, có, 5, chữ cái, ,, thỉnh thoản..."


## 5.Removing Stop Words

In [ ]:
# Removing Stop Words
stop_words = stopwords.words("english")
for i in range(len(df['Question'])):
  rs_result = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)] for tokens in df['rs_result'][i]]
  df['rs_result'][i]=rs_result
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],"[[Nghe, thông, mình, phải, không]]"
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],"[[Robot, thất tình, có thể, không]]"
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],"[[Robot, mình, chỉ số, bao nhiêu]]"
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],"[[Robot, thích, chơi, búp bê, không]]"
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],"[[Theo, robot, cộng, bằng]]"
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],"[[trước, trứng, trước]]"
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],"[[trước, trứng, trước]]"
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],"[[Nhiệt độ, nóng chảy, thủy ngân, bao nhiêu]]"
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],"[[chữ cái, thỉnh thoảng, chữ cái, không, bao g..."


In [ ]:
df['rs_result'][i][0]

['What']

# The Word representation methods do not use Word2vec.

In [ ]:
def chatbot(sentence):
  max=0
  temp=0
  tempid=0
  Question_Tokens=classify(sentence)
  print("Question",Question_Tokens,sentence)
  Test=word_tokenize(sentence)
  for i in range(len(df['Question_Tokens'])):
    try:
      if(Question_Tokens[0]==df['Question_Tokens'][i][0]):
        for sentence1 in df['rs_result'][i][0]:
          for sentence2 in Test:
            if sentence1==sentence2:
              temp+=1
            if max < temp:
              tempid=i
              max=temp
    except Exception:
      pass
  print("Aws:",df['Answer'][tempid])

## Test

In [ ]:
chatbot('Bạn đang học ngành gì?')

Question ['chinh_tri_xa_hoi'] Bạn đang học ngành gì?
Aws:  chị học Công nghệ thông tin á em 


In [ ]:
chatbot('Bạn cho mình hỏi bạn SDT của bạn ?')

Question ['doi_song'] Bạn cho mình hỏi bạn SDT của bạn ?
Aws:  có 


In [ ]:
chatbot('bạn đã có người yêu chưa')

Question ['doi_song'] bạn đã có người yêu chưa
Aws:  Mình chưa có người yêu nên không biết mùi cắm sừng là gì hết. 


In [ ]:
chatbot('Theo robot thì 1 cộng 1 bằng mấy ?')

Question ['doi_song'] Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?
Aws:  Hơn m được rồi phải không 


# Word2Vec

In [ ]:
from gensim.models import Word2Vec

bigger_list=[]
for i in df['Question']:
    li = list(i.split(" "))
    bigger_list.append(li)
#structure of data to be taken by the model.word2vec
print("Data format for the overall list:",bigger_list)
#custom data is fed to machine for further processing
model = Word2Vec(bigger_list, min_count=1,size=300,workers=4)
print(model)

Data format for the overall list: [['Nghe', 'nói', 'thông', 'mình', 'lắm', '', 'phải', 'không?', ''], ['Robot', 'ơi', 'tui', 'thất', 'tình', 'rồi,', 'bạn', 'có', 'thể', 'làm', 'bạn', 'gái', 'tui', 'không', '?', ''], ['Robot', 'cho', 'mình', 'hỏi', 'IQ', 'của', 'bạn', 'chỉ', 'số', 'bao', 'nhiêu', 'vậy', '?', ''], ['Robot', 'có', 'thích', 'chơi', 'búp', 'bê', 'không', '?', ''], ['Theo', 'robot', 'thì', '1', 'cộng', '1', 'bằng', 'mấy', '?', ''], ['nay', 'tôi', 'buồn', 'quá', ''], ['bạn', 'biết', 'hát', 'hay', 'không?', ''], ['bạn', 'biết', 'hát', 'hay', 'không?', ''], ['bạn', 'biết', 'gì', 'về', 'thế', 'giới', 'xung', 'quanh?', ''], ['1', 'với', '1', 'là?', ''], ['1', 'với', '1', 'là?', ''], ['Bạn', 'đang', 'làm', 'gì?', ''], ['Vậy', '1+2010', 'bằng', 'bao', 'nhiêu', ''], ['Bạn', 'còn', 'biết', 'làm', 'những', 'gì', ''], ['Cậu', 'có', 'biết', 'đêm', 'qua', 'tôi', 'đã', 'nằm', 'mơ', 'thấy', 'gì', 'không?', ''], ['Tớ', 'có', 'thể', 'phá', 'hoại', 'cậu', 'không?', ''], ['Tôi', '

In [ ]:
!wget https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin
#!wget https://github.com/nthanhkhang/Natural-Language-Processing/blob/main/Data/DataChatbot.zip
model = gensim.models.KeyedVectors.load_word2vec_format('baomoi.model.bin', binary=True)

--2021-04-25 02:14:12--  https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.model.bin
Resolving thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)... 52.219.68.212
Connecting to thiaisotajppub.s3-ap-northeast-1.amazonaws.com (thiaisotajppub.s3-ap-northeast-1.amazonaws.com)|52.219.68.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 708212586 (675M) [application/macbinary]
Saving to: ‘baomoi.model.bin’

baomoi.model.bin    100%[===================>] 675.40M  31.9MB/s    in 23s     

2021-04-25 02:14:35 (29.5 MB/s) - ‘baomoi.model.bin’ saved [708212586/708212586]



In [ ]:
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],"[[Nghe, thông, mình, phải, không]]"
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],"[[Robot, thất tình, có thể, không]]"
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],"[[Robot, mình, chỉ số, bao nhiêu]]"
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],"[[Robot, thích, chơi, búp bê, không]]"
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],"[[Theo, robot, cộng, bằng]]"
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],"[[trước, trứng, trước]]"
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],"[[trước, trứng, trước]]"
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],"[[Nhiệt độ, nóng chảy, thủy ngân, bao nhiêu]]"
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],"[[chữ cái, thỉnh thoảng, chữ cái, không, bao g..."


In [ ]:
def listToString(s): 
    str1 = ""  
    for ele in s: 
      str1 += ele+' '
    return str1    
for i in range(len(df['rs_result'])):
  df['rs_result'][i]=listToString(df['rs_result'][i][0])
df

,Question,Answer,Question_Tokens,rs_result
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],Nghe thông mình phải không
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],Robot thất tình có thể không
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],Robot mình chỉ số bao nhiêu
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],Robot thích chơi búp bê không
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],Theo robot cộng bằng
...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],trước trứng trước
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],trước trứng trước
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],Nhiệt độ nóng chảy thủy ngân bao nhiêu
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],chữ cái thỉnh thoảng chữ cái không bao giờ chữ...


In [ ]:
A=[]
sentence='cho mình hỏi bạn tên gì'
sentence=word_tokenize(sentence)
for i in sentence:
  A.append(model.similar_by_word(i))
print(A)

[[('Được', 0.45118680596351624), ('Ít', 0.26197636127471924), ('tiết_chế_bản_thân', 0.25506672263145447), ('đối_với', 0.24585318565368652), ('có_nghĩa', 0.24196317791938782), ('trung_quốccho', 0.2415129691362381), ('chho', 0.23434969782829285), ('tin_tưởng', 0.2299114316701889), ('trương_công_lý', 0.2273520827293396), ('rằng_biết', 0.22626499831676483)], [('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498), ('bạn', 0.5670677423477173), ('cô', 0.540347695350647), ('nó', 0.5179349184036255), ('anh', 0.4915458559989929), ('tớ', 0.48625844717025757), ('ba_mẹ', 0.4696437120437622)], [('trả_lời', 0.624562680721283), ('nói', 0.5494940876960754), ('hỏi_nhỏ', 0.5434095859527588), ('thắc_mắc', 0.5036576390266418), ('giải_thích', 0.5008779764175415), ('hỏi_thăm_dò', 0.4940515458583832), ('hỏi_đi_hỏi_lại', 0.4924977421760559), ('hỏi_thăm', 0.48845016956329346), ('căn_vặn', 0.4841855466365814), ('hỏi_hỏi', 0.4776047468185425)], [('em

In [ ]:
similar=[]
sentence='cho mình hỏi bạn tên gì'
sentence=word_tokenize(sentence, format="text")
sentence=word_tokenize(sentence)
for i in sentence:
  similar.append(model.similar_by_word('mình'))
print(similar)
#print('',similar[3][0],similar[3][1])

[[('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498), ('bạn', 0.5670677423477173), ('cô', 0.540347695350647), ('nó', 0.5179349184036255), ('anh', 0.4915458559989929), ('tớ', 0.48625844717025757), ('ba_mẹ', 0.4696437120437622)], [('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498), ('bạn', 0.5670677423477173), ('cô', 0.540347695350647), ('nó', 0.5179349184036255), ('anh', 0.4915458559989929), ('tớ', 0.48625844717025757), ('ba_mẹ', 0.4696437120437622)], [('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498), ('bạn', 0.5670677423477173), ('cô', 0.540347695350647), ('nó', 0.5179349184036255), ('anh', 0.4915458559989929), ('tớ', 0.48625844717025757), ('ba_mẹ', 0.4696437120437622)], [('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498

In [ ]:
print(model.similar_by_word('mình'))

[('tôi', 0.6837077140808105), ('họ', 0.6683597564697266), ('bản_thân', 0.6430788040161133), ('em', 0.5692948698997498), ('bạn', 0.5670677423477173), ('cô', 0.540347695350647), ('nó', 0.5179349184036255), ('anh', 0.4915458559989929), ('tớ', 0.48625844717025757), ('ba_mẹ', 0.4696437120437622)]


In [ ]:
df['rs_result_w2v']=df['Question'].apply(len)

In [ ]:
for i in range(len(df)):
  sentences=[]
  similar=[]
  A=[]
  sentence=df['rs_result'][i]
  sentence=word_tokenize(sentence)
  for s in sentence:
    sentences.append(word_tokenize(s, format="text"))
  print('Test',i,sentences)
  for s in sentences:
    #print(i)
    try:
      A.append(model.similar_by_word(s))
    except Exception:
      pass
  for j in range(len(A)):
    similar.append(A[j][0][1])
  # print(similar)
  df['rs_result_w2v'][i]=str(similar)
  print(df['rs_result_w2v'][i])

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Test 3355 ['mình', 'nâng cấp', 'laptop', 'không']
[0.6837077140808105, 0.8329999446868896, 0.7715991735458374]
Test 3356 ['mình', 'định', 'laptop', 'không']
[0.6837077140808105, 0.4293029308319092, 0.8329999446868896, 0.7715991735458374]
Test 3357 ['mình', 'quần_áo', 'không']
[0.6837077140808105, 0.8750599026679993, 0.7715991735458374]
Test 3358 ['Điện_thoại']
[0.6517419815063477]
Test 3359 ['Điện_thoại']
[0.6517419815063477]
Test 3360 ['quần', 'không', 'shop']
[0.7391069531440735, 0.7715991735458374, 0.7813281416893005]
Test 3361 ['chất_liệu']
[0.6355317831039429]
Test 3362 ['quần', 'size']
[0.7391069531440735, 0.833787202835083]
Test 3363 ['nặng']
[0.5252716541290283]
Test 3364 ['vàng', 'không']
[0.4744075536727905, 0.7715991735458374]
Test 3365 ['bao_nhiêu']
[0.5008718967437744]
Test 3366 ['shop', 'mình', 'Iphone', 'không']
[0.7813281416893005, 0.6837077140808105, 0.7715991735458374]
Test 3367 ['shop', 'mình', 'Iphone', 'không

In [ ]:
df.head()

,Question,Answer,Question_Tokens,rs_result,rs_result_w2v
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],Nghe thông mình phải không,"[0.36375415325164795, 0.6837077140808105, 0.49..."
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],Robot thất tình có thể không,"[0.5952091813087463, 0.5237374305725098, 0.771..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],Robot mình chỉ số bao nhiêu,"[0.6837077140808105, 0.5008718967437744]"
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],Robot thích chơi búp bê không,"[0.6371036767959595, 0.5997104644775391, 0.538..."
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],Theo robot cộng bằng,"[0.7083206176757812, 0.3925487697124481, 0.463..."


In [ ]:
for i in range(len(df)):
  s=df['rs_result_w2v'][i]
  df['rs_result_w2v'][i]=s.translate({ord('['): None})
  s=df['rs_result_w2v'][i]
  df['rs_result_w2v'][i]=s.translate({ord(']'): None})

In [ ]:
df['rs_result_w2v'][1]

'0.5952091813087463, 0.5237374305725098, 0.7715991735458374'

In [ ]:
X=0
for i in range(len(df)):
  txt=df['rs_result_w2v'][i]
  x = txt.split(", ")
  df['rs_result_w2v'][i]=x

In [ ]:
df['rs_result_w2v'][0][1]

'0.6837077140808105'

In [ ]:
df

,Question,Answer,Question_Tokens,rs_result,rs_result_w2v
0,Nghe nói thông mình lắm phải không?,Hơn m được rồi phải không,[chinh_tri_xa_hoi],Nghe thông mình phải không,"[0.36375415325164795, 0.6837077140808105, 0.49..."
1,"Robot ơi tui thất tình rồi, bạn có thể làm bạn...","Ơ, chia buồn cùng bạn, bạn hãy sớm vượt qua đ...",[doi_song],Robot thất tình có thể không,"[0.5952091813087463, 0.5237374305725098, 0.771..."
2,Robot cho mình hỏi IQ của bạn chỉ số bao nhiêu...,Mình không ghi được vì vô cực tràn bộ nhớ,[khoa_hoc],Robot mình chỉ số bao nhiêu,"[0.6837077140808105, 0.5008718967437744]"
3,Robot có thích chơi búp bê không ?,Mình chẳng những chơi búp bê mà còn quất khủn...,[khoa_hoc],Robot thích chơi búp bê không,"[0.6371036767959595, 0.5997104644775391, 0.538..."
4,Theo robot thì 1 cộng 1 bằng mấy ?,"Từ thời xa xưa, 1 ngón với 1 ngón sẽ là 2 ngón",[khoa_hoc],Theo robot cộng bằng,"[0.7083206176757812, 0.3925487697124481, 0.463..."
...,...,...,...,...,...
5850,Con gà có trước hay quả trứng có trước ?,Kệ nó hỏi làm gì,[chinh_tri_xa_hoi],trước trứng trước,"[0.7365881204605103, 0.5643246173858643, 0.736..."
5851,Con chó có trước hay quả trứng có trước?,Quả trứng có trước,[doi_song],trước trứng trước,"[0.7365881204605103, 0.5643246173858643, 0.736..."
5852,Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?,âm 38.88 độ C. Gà,[doi_song],Nhiệt độ nóng chảy thủy ngân bao nhiêu,"[0.5878803730010986, 0.7058447599411011, 0.500..."
5853,"Cái gì chỉ có 5 chữ cái, thỉnh thoảng có 11 ch...",Cái gì lạ vậy ?,[doi_song],chữ cái thỉnh thoảng chữ cái không bao giờ chữ...,"[0.6071650385856628, 0.5057944059371948, 0.929..."


In [ ]:
sentence='Con gà có trước hay quả trứng có trước ?'
sentence=word_tokenize(sentence, format="text")
sentence=word_tokenize(sentence)
re_result=[]
re_result.append(sentence)
stop_words = stopwords.words("english")
rs_result = [[t for t in tokens if (t not in stop_words) and (3 < len(t.strip()) < 15)] for tokens in re_result]
rs_result=rs_result[0]

In [ ]:
print(rs_result)

['trước', 'trứng', 'trước']


In [ ]:
# locate=0
def chatbot_w2v(sentence):
  sentences=[]
  similar=[]
  A=[]
  temp=0
  max=0
  sentence=word_tokenize(sentence)
  for s in sentence:
    sentences.append(word_tokenize(s, format="text"))
  print(sentences)
  for s in sentences:
    #print(i)
    try:
      A.append(model.similar_by_word(s))
    except Exception:
      pass
  for j in range(len(A)):
    similar.append(A[j][0][1])

  for i in range(len(df)):
    temp=0
    try:
      for j in range(len(df['rs_result_w2v'][i])):
        for k in range(len(similar)):
          if(float(similar[k])==float(df['rs_result_w2v'][i][j])):
            # print(similar[k])
            # print(df['rs_result_w2v'][i][j])
            # print('oki')
            temp+=1
          if max <= temp:
            max=temp
            locate=i
            # print(df['Answer'][i])
    except Exception:
      pass
  print('Answer:',df['Answer'][locate])
  return locate

## Test

In [ ]:
chatbot_w2v("Robot có thích chơi búp bê không?")

['Robot', 'có', 'thích', 'chơi', 'búp_bê', 'không', '?']
Answer:  Mình chẳng những chơi búp bê mà còn quất khủng long siêu nhân gao 


3

In [ ]:
chatbot_w2v("Con chó có trước hay quả trứng có trước")

['Con', 'chó', 'có', 'trước', 'hay', 'quả', 'trứng', 'có', 'trước']
Answer:  Quả trứng có trước 


5851

In [ ]:
chatbot_w2v("Theo robot thì 1 cộng 1 bằng mấy")

['Theo', 'robot', 'thì', '1', 'cộng', '1', 'bằng', 'mấy']
Answer:  đáp án tổng cộng của bạn là 11 nhé . Bạn còn hỏi gì nữa không? 


66

In [ ]:
chatbot_w2v("Nhiệt độ nóng chảy của thủy ngân là bao nhiêu ?")

['Nhiệt_độ', 'nóng_chảy', 'của', 'thủy_ngân', 'là', 'bao_nhiêu', '?']
Answer:  âm 38.88 độ C. Gà 


5852